In [1]:
import datetime
from IPython.display import display
import itertools
import concurrent.futures
from matplotlib import pyplot as plt
import multiprocessing
import numpy as np
import pandas as pd
import random
from scipy.optimize import minimize
from scipy import linalg
# import seaborn as sns
import statistics as stats
import time
import warnings
warnings.filterwarnings('ignore')

# # organize DataSeries
# date = data.index
# tLen = len(data)
# time = np.linspace(0, tLen-1, tLen)
# close = [data["Adj Close"][i] for i in range(len(data["Adj Close"]))]
# DataSeries = [time, close]

# matrix helpers
def _yi(price_series):
    return [np.log(p) for p in price_series]

def _fi(tc, m, time_series):
    return [np.power((tc - t), m) if str(np.power((tc - t), m)) != 'nan' else 0 for t in time_series]
     
def _gi(tc, m, w, time_series):
    return [np.power((tc - t), m) * np.cos(w * np.log(tc - t)) if str(np.power((tc - t), m) * np.cos(w * np.log(tc - t))) != 'nan' else 0 for t in time_series]

def _hi(tc, m, w, time_series):
    return [np.power((tc - t), m) * np.sin(w * np.log(tc - t)) if str(np.power((tc - t), m) * np.sin(w * np.log(tc - t))) != 'nan' else 0 for t in time_series]

def _fi_pow_2(tc, m, time_series):
    return np.power(_fi(tc, m, time_series), 2)

def _gi_pow_2(tc, m, w, time_series):
    return np.power(_gi(tc, m, w, time_series), 2)

def _hi_pow_2(tc, m, w, time_series):
    return np.power(_hi(tc, m, w, time_series), 2)

def _figi(tc, m, w, time_series):
    return np.multiply(_fi(tc, m, time_series), _gi(tc, m, w, time_series))

def _fihi(tc, m, w, time_series):
    return np.multiply(_fi(tc, m, time_series), _hi(tc, m, w, time_series))

def _gihi(tc, m, w, time_series):
    return np.multiply(_gi(tc, m, w, time_series), _hi(tc, m, w, time_series))

def _yifi(tc, m, time_series, price_series):
    return np.multiply(_yi(price_series), _fi(tc, m, time_series))

def _yigi(tc, m, w, time_series, price_series):
    return np.multiply(_yi(price_series), _gi(tc, m, w, time_series))

def _yihi(tc, m, w, time_series, price_series):
    return np.multiply(_yi(price_series), _hi(tc, m, w, time_series))

In [2]:
data = pd.read_csv("../data/backtest-2.csv", index_col="date")
data.fillna(method='ffill')

# new = data[['AAPL']].copy()
# new.plot(figsize=(14,8))

print(len(data))
display(data.head())
display(data.tail())

254


,A,AA,AABA,AAC,AAL,AAMC,AAME,AAN,AAOI,AAON,...,ZS,ZSAN,ZTO,ZTR,ZTS,ZUMZ,ZUO,ZYME,ZYNE,ZYXI
date,,,,,,,,,,,,,,,,,,,,,
2018-03-12,70.0607,48.36,77.56,12.27,55.9142,63.9,3.4040,48.7580,30.95,37.8506,...,NaN,15.39,15.0003,9.8711,83.8821,19.9,NaN,10.49,10.53,4.09
2018-03-13,70.0309,48.59,75.97,11.94,55.6470,65.0,3.2798,48.2194,29.51,38.2965,...,NaN,16.70,15.0792,9.8623,84.0609,20.0,NaN,10.67,10.39,4.15
2018-03-14,69.2779,46.57,77.20,12.04,54.7069,58.6,3.2748,48.2095,29.12,38.1479,...,NaN,15.80,15.0299,9.8623,83.8224,19.6,NaN,10.68,10.06,4.00
2018-03-15,69.5355,46.93,79.42,12.70,54.5980,60.9,3.2798,47.7208,26.79,38.4451,...,NaN,15.50,15.1285,9.8623,84.1603,20.0,NaN,11.05,10.17,3.93
2018-03-16,69.2581,47.23,79.90,12.87,54.8256,62.9,3.1804,48.1895,26.78,38.7424,...,33.0,17.44,15.2371,9.8800,84.0709,22.1,NaN,11.86,9.42,4.00


,A,AA,AABA,AAC,AAL,AAMC,AAME,AAN,AAOI,AAON,...,ZS,ZSAN,ZTO,ZTR,ZTS,ZUMZ,ZUO,ZYME,ZYNE,ZYXI
date,,,,,,,,,,,,,,,,,,,,,
2019-03-08,77.98,26.82,71.75,2.31,31.91,35.50,2.5800,52.78,12.88,41.59,...,60.54,4.76,20.17,10.45,92.41,24.61,23.04,15.85,4.78,4.8600
2019-03-11,79.30,27.52,73.47,2.56,32.05,34.77,2.5201,53.21,13.30,41.37,...,63.34,5.24,20.13,10.50,94.73,24.61,23.55,16.72,4.86,4.6500
2019-03-12,79.93,28.44,73.59,2.75,30.92,34.30,2.5200,52.59,13.49,41.57,...,63.65,5.24,20.01,10.54,95.75,24.58,23.52,16.65,5.11,4.7600
2019-03-13,80.13,28.98,73.64,2.74,31.84,35.65,2.7000,53.09,14.08,41.94,...,64.80,5.10,19.29,10.49,96.93,25.33,23.63,16.74,5.10,4.8000
2019-03-14,80.61,27.89,73.47,2.64,32.13,36.35,2.7000,53.26,13.84,41.19,...,66.94,5.07,19.12,10.67,96.20,23.68,23.50,15.75,4.90,4.5588


In [3]:
# revised version of the LPPL without φ
# found on page 11 as equation (13)
def lppl(t, tc, m, w, a, b, c1, c2):
    return a + np.power(tc - t, m) * (b + ((c1 * np.cos(w * np.log(tc - t))) + (c2 * np.sin(w * np.log(tc - t)))))

# finds the least square difference
def func_restricted(x, *args):
    tc = x[0]
    m  = x[1]
    w  = x[2]
    
    data_series = args[0]

    lin_vals = matrix_equation(tc, m, w, data_series)
 
    a  = lin_vals[0] 
    b  = lin_vals[1]
    c1 = lin_vals[2] 
    c2 = lin_vals[3]
    
    delta = [lppl(t, tc, m, w, a, b, c1, c2) for t in data_series[0]]
    delta = np.subtract(delta, data_series[1])
    delta = np.power(delta, 2)
    return np.sum(delta)

            
# solve the matrix equation
def matrix_equation(tc, m, w, data_series):
    time_series = data_series[0]
    price_series = data_series[1]
    N  = len(price_series)
    
    fi = np.sum(_fi(tc, m, time_series))
    gi = np.sum(_gi(tc, m, w, time_series))
    hi = np.sum(_hi(tc, m, w, time_series))

    fi_pow_2 = np.sum(_fi_pow_2(tc, m, time_series))
    gi_pow_2 = np.sum(_gi_pow_2(tc, m, w, time_series))
    hi_pow_2= np.sum(_hi_pow_2(tc, m, w, time_series))

    figi = np.sum(_figi(tc, m, w, time_series))
    fihi = np.sum(_fihi(tc, m, w, time_series))
    gihi = np.sum(_gihi(tc, m, w, time_series))

    yi = np.sum(_yi(price_series))
    yifi = np.sum(_yifi(tc, m, time_series, price_series))
    yigi = np.sum(_yigi(tc, m, w, time_series, price_series))
    yihi = np.sum(_yihi(tc, m, w, time_series, price_series))
    
    
    matrix_1 = np.matrix([
        [N,  fi,       gi,       hi      ],
        [fi, fi_pow_2, figi,     fihi    ],
        [gi, figi,     gi_pow_2, gihi    ],
        [hi, fihi,     gihi,     hi_pow_2]
    ])
    
    matrix_2 = np.matrix([
        [yi],
        [yifi],
        [yigi],
        [yihi]
    ])

    try:
        product = linalg.solve(matrix_1, matrix_2)
        product_list = product.tolist()
        return [i[0] for i in product_list]
    except Exception as e:
        print(e)
        print(data_series)
        print(matrix_1)
        print(matrix_2)
        

In [4]:
def compute_ds_lppls_confidence(n, s):
    
    tLen = 250-(n*5)
    print('length of data: {}'.format(tLen))
    tradings_days_data = data.tail(tLen)
    time = np.linspace(0, tLen-1, tLen)
    price = [tradings_days_data[s][i] for i in range(len(tradings_days_data[s]))]
    data_series = [time, price]

    found_solution = False

    while not found_solution:

        # set limits for non-linear params
        limits = (
            [tLen-(tLen*0.2), tLen+(tLen*0.2)],    # Critical Time + or - .2
            [0.1, 0.9],                            # m : 0.1 ≤ m ≤ 0.9
            [6, 13],                               # ω : 6 ≤ ω ≤ 13     
        )

        # randomly choose vals for non-linear params 
        non_lin_vals = [random.uniform(a[0], a[1]) for a in limits]

        tc = non_lin_vals[0]
        m  = non_lin_vals[1] 
        w  = non_lin_vals[2]

        # params to pass to scipy.optimize
        seed = [tc, m, w]

        # scipy optimize minimize
        try:
            
            
            cofs = minimize(fun=func_restricted, x0=seed, args=(data_series), method='Nelder-Mead')

            if cofs.success:
                print('minimize ran succsess')
                # determine if it falls in range:

                tc = cofs.x[0]
                m =  cofs.x[1]
                w =  cofs.x[2]

                tc_in_range = tLen-(tLen*0.05) < tc < tLen+(tLen*0.1)
                m_in_range  = 0.01 < m < 1.2
                w_in_range  = 2 < w < 25
                n_oscillation = ((w/2)*np.log(abs((tc - (n*5))/(tLen)))) > 2.5

                if (tc_in_range and m_in_range and w_in_range and n_oscillation):
                    ds_lppls_confidence = True
                else: 
                    ds_lppls_confidence = False

                return {
                    'ds_lppls_confidence': ds_lppls_confidence,
                    'cof': cofs.x,
                }
        except Exception as e:
            print('minimize failed: {}'.format(e))
            print('tc: {}, m: {}, w: {}'.format(tc,m,w))
#             print('seed: {}'.format(seed))
#             print('data_series:')
#             print(data_series)
            
    
# def main(time_range, s):
#     solutions = []
#     for i in range(time_range):
        
        
#         #     with concurrent.futures.ProcessPoolExecutor() as executor:
#         #         for foo in executor.map(compute_ds_lppls_confidence, time_range, s):
#         #             solutions.append(foo)

#         foo = compute_ds_lppls_confidence(i, s)
#         solutions.append(foo)
        
#     true_count = 0
#     total_count = len(solutions)
#     for i in solutions:
#         if i["ds_lppls_confidence"] == True:
#             true_count = true_count + 1

# #     print("{}/{}={}".format(true_count,total_count,true_count/total_count))
#     return (true_count/total_count)


In [5]:
SYMBOL = 'AAPL'
duration_of_fit = []
ds_lppls = []
upperbound = 250 # ~1 year (in trading days)
lowerbound = 125 # ~7 months (in trading days)
number_of_fitting_windows = int((upperbound - lowerbound)/5)
print('number of fitting windows: {}'.format(number_of_fitting_windows))

# this should be parallelized 

for i in range(number_of_fitting_windows):
    print('fitting window: {}'.format(i))
    start = time.time()
    ds_lppl = compute_ds_lppls_confidence(i, SYMBOL)
    end = time.time()
    duration_of_fit.append(end - start)
    ds_lppls.append({SYMBOL: ds_lppl})
    
    print(duration_of_fit)
#     print(ds_lppls)



number of fitting windows: 25
fitting window: 0
length of data: 250
minimize ran succsess
[27.944775342941284]
fitting window: 1
length of data: 245
minimize ran succsess
[27.944775342941284, 7.047161102294922]
fitting window: 2
length of data: 240
minimize ran succsess
[27.944775342941284, 7.047161102294922, 9.13986873626709]
fitting window: 3
length of data: 235
minimize ran succsess
[27.944775342941284, 7.047161102294922, 9.13986873626709, 25.676658868789673]
fitting window: 4
length of data: 230
minimize ran succsess
[27.944775342941284, 7.047161102294922, 9.13986873626709, 25.676658868789673, 8.171267986297607]
fitting window: 5
length of data: 225
minimize ran succsess
[27.944775342941284, 7.047161102294922, 9.13986873626709, 25.676658868789673, 8.171267986297607, 22.742730617523193]
fitting window: 6
length of data: 220
minimize ran succsess
[27.944775342941284, 7.047161102294922, 9.13986873626709, 25.676658868789673, 8.171267986297607, 22.742730617523193, 6.72235107421875]
fitt

Matrix is singular.
[array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
        11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
        22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,
        33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,
        44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,
        55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,
        66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,
        77.,  78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,
        88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,
        99., 100., 101., 102., 103., 104., 105., 106., 107., 108., 109.,
       110., 111., 112., 113., 114., 115., 116., 117., 118., 119., 120.,
       121., 122., 123., 124., 125., 126., 127., 128., 129., 130., 131.,
       132., 133., 134., 135., 136., 137., 138., 139., 140., 141., 142.,
       143., 144.]), [208.6142

Matrix is singular.
[array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
        11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
        22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,
        33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,
        44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,
        55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,
        66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,
        77.,  78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,
        88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,
        99., 100., 101., 102., 103., 104., 105., 106., 107., 108., 109.,
       110., 111., 112., 113., 114., 115., 116., 117., 118., 119., 120.,
       121., 122., 123., 124., 125., 126., 127., 128., 129., 130., 131.,
       132., 133., 134., 135., 136., 137., 138., 139., 140., 141., 142.,
       143., 144.]), [208.6142

Matrix is singular.
[array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
        11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
        22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,
        33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,
        44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,
        55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,
        66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,
        77.,  78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,
        88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,
        99., 100., 101., 102., 103., 104., 105., 106., 107., 108., 109.,
       110., 111., 112., 113., 114., 115., 116., 117., 118., 119., 120.,
       121., 122., 123., 124., 125., 126., 127., 128., 129., 130., 131.,
       132., 133., 134., 135., 136., 137., 138., 139., 140., 141., 142.,
       143., 144.]), [208.6142

Matrix is singular.
[array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
        11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
        22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,
        33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,
        44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,
        55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,
        66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,
        77.,  78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,
        88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,
        99., 100., 101., 102., 103., 104., 105., 106., 107., 108., 109.,
       110., 111., 112., 113., 114., 115., 116., 117., 118., 119., 120.,
       121., 122., 123., 124., 125., 126., 127., 128., 129., 130., 131.,
       132., 133., 134., 135., 136., 137., 138., 139., 140., 141., 142.,
       143., 144.]), [208.6142

...

Matrix is singular.
[array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
        11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
        22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,
        33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,
        44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,
        55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,
        66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,
        77.,  78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,
        88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,
        99., 100., 101., 102., 103., 104., 105., 106., 107., 108., 109.,
       110., 111., 112., 113., 114., 115., 116., 117., 118., 119., 120.,
       121., 122., 123., 124., 125., 126., 127., 128., 129., 130., 131.,
       132., 133., 134., 135., 136., 137., 138., 139., 140., 141., 142.,
       143., 144.]), [208.6142

Matrix is singular.
[array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
        11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
        22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,
        33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,
        44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,
        55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,
        66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,
        77.,  78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,
        88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,
        99., 100., 101., 102., 103., 104., 105., 106., 107., 108., 109.,
       110., 111., 112., 113., 114., 115., 116., 117., 118., 119., 120.,
       121., 122., 123., 124., 125., 126., 127., 128., 129., 130., 131.,
       132., 133., 134., 135., 136., 137., 138., 139., 140., 141., 142.,
       143., 144.]), [208.6142

Matrix is singular.
[array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
        11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
        22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,
        33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,
        44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,
        55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,
        66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,
        77.,  78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,
        88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,
        99., 100., 101., 102., 103., 104., 105., 106., 107., 108., 109.,
       110., 111., 112., 113., 114., 115., 116., 117., 118., 119., 120.,
       121., 122., 123., 124., 125., 126., 127., 128., 129., 130., 131.,
       132., 133., 134., 135., 136., 137., 138., 139., 140., 141., 142.,
       143., 144.]), [208.6142

Matrix is singular.
[array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
        11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
        22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,
        33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,
        44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,
        55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,
        66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,
        77.,  78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,
        88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,
        99., 100., 101., 102., 103., 104., 105., 106., 107., 108., 109.,
       110., 111., 112., 113., 114., 115., 116., 117., 118., 119., 120.,
       121., 122., 123., 124., 125., 126., 127., 128., 129., 130., 131.,
       132., 133., 134., 135., 136., 137., 138., 139., 140., 141., 142.,
       143., 144.]), [208.6142

KeyboardInterrupt: 

In [ ]:
print(ds_lppls)

In [ ]:
for i in range(number_of_fitting_windows):
    tLen = 250-(i*5)
    DataSeries = data.tail(tLen)
    time2 = DataSeries.index.tolist()
    time2 = np.linspace(0, len(time2)-1, len(time2))
    price = DataSeries[SYMBOL].tolist()
    DataSeries = [time2, price]

    tc = ds_lppls[i][SYMBOL]['cof'][0]
    m =  ds_lppls[i][SYMBOL]['cof'][1]
    w =  ds_lppls[i][SYMBOL]['cof'][2]

    lin_vals = matrix_equation(tc, m, w, DataSeries)

    a  = lin_vals[0] 
    b  = lin_vals[1]
    c1 = lin_vals[2] 
    c2 = lin_vals[3]
    lppl_fit = [lppl(t, tc, m, w, a, b, c1, c2) for t in DataSeries[0]]
    price_data = DataSeries[1]

    data2 = pd.DataFrame({
        'Date': DataSeries[0],
        'LPPLS Fit': lppl_fit,
        SYMBOL: np.log(price_data),
    })
    data2 = data2.set_index('Date')
    data2.plot(figsize=(14,8))

In [ ]:
print(stats.mean(duration_of_fit))
print(sum(duration_of_fit))